In [1]:
import nltk
from nltk.corpus import stopwords
import gensim
import string

In [2]:
import pandas as pd
import numpy as np

In [8]:
reviews=pd.read_csv('imdb_sentiment.csv')
reviews.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
reviews.sentiment.value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [10]:
reviews_combined = " ".join(reviews.review.values)

In [11]:
all_terms = []
fdist = {}
all_terms = reviews_combined.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [12]:
from string import punctuation
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")
stop_updated = stop_nltk + ["...","..","!!"] +["phone",
                                          "mobile","lenovo","k8",
                                               "note","amazon","n't"]

In [13]:
reviews_updated1 = [term for term in all_terms \
                    if term not in stop_updated \
and term not in list(punctuation) and len(term)>2]

In [14]:
from nltk.stem import SnowballStemmer
stemmer_s = SnowballStemmer("english")

In [15]:
reviews_updated_stem = [ stemmer_s.stem(word) for word in reviews_updated1]

In [16]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

In [17]:
reviews_updated_lemm = [ lemm.lemmatize(word) for word in reviews_updated1]

In [20]:
from nltk.tokenize import word_tokenize

In [21]:
def  clean_txt(sent):
    tokens = word_tokenize(sent.lower())
    stemmed = [stemmer_s.stem(term) for term in tokens 
               if term not in stop_updated and
               term not in list(punctuation) and len(term) > 2] 
    res = " ".join(stemmed)
    return res

In [22]:
reviews['clean_review'] = reviews.review.apply(clean_txt)

In [23]:
reviews.head()

,review,sentiment,clean_review
0,"A very, very, very slow-moving, aimless movie ...",0,slow-mov aimless movi distress drift young man
1,Not sure who was more lost - the flat characte...,0,sure lost flat charact audienc near half walk
2,Attempting artiness with black & white and cle...,0,attempt arti black white clever camera angl mo...
3,Very little music or anything to speak of.,0,littl music anyth speak
4,The best scene in the movie was when Gerardo i...,1,best scene movi gerardo tri find song keep run...


In [ ]:
#from string import punctuation
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
#stop_nltk=stopwords.words("english")
#d=stop_nltk+["...","..","!!"]+['would','could','told','subject']

#def clean_txt(sent):
    #tokens=word_tokenize(sent.lower())
   # rev=[term for term in tokens if term not in d and term not in list(punctuation) and len(term)>2]
    #res=" ".join(rev)
    #return res

In [27]:
docs_vectors=pd.DataFrame()
for doc in reviews.clean_review:
    temp=pd.DataFrame()
    words=doc.split(' ')
    #print(words)
    for word in words:
        try:
            word2vec=embeddings[word]   ## gives vector of length
            temp=temp.append(pd.Series(word2vec),ignore_index=True)
        except:
            pass
    doc_vector=temp.mean()
    docs_vectors=docs_vectors.append(doc_vector,ignore_index=True)
docs_vectors.head()     

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.142660,0.123922,-0.145142,0.073232,-0.106445,-0.007273,-0.106486,-0.023753,0.012431,0.036760,...,0.033691,0.065633,-0.089193,-0.128990,-0.094889,0.037313,0.005534,-0.044490,0.041667,0.088338
1,0.065959,-0.009280,-0.025065,0.155579,-0.042419,-0.047282,0.006429,-0.119019,0.115560,0.089111,...,-0.055379,0.059530,-0.132243,0.024272,-0.048584,-0.047811,-0.057088,-0.063049,0.037214,-0.136800
2,0.110306,0.027736,0.005709,0.051640,-0.088618,-0.001378,0.034108,0.009286,0.000392,0.049298,...,-0.025678,0.041327,-0.117480,0.011088,-0.096287,-0.023344,-0.009563,-0.034526,-0.009094,-0.017124
3,-0.002197,-0.085286,-0.022542,0.060818,-0.025187,0.077962,0.055176,-0.128011,0.008138,-0.040202,...,0.066650,0.088216,-0.051270,-0.019450,0.040283,-0.042643,0.042643,-0.033122,0.123830,-0.012939
4,0.002970,0.004822,0.009481,0.044481,-0.052585,-0.027764,-0.032145,-0.073251,0.070543,-0.001460,...,-0.012163,0.067600,-0.100895,0.000629,-0.022108,-0.079359,0.015357,-0.069292,0.014838,0.010417


In [4]:
google_model='GoogleNews-vectors-negative300.bin'
embeddings=gensim.models.KeyedVectors.load_word2vec_format(google_model,binary=True)

In [28]:
docs_vectors=docs_vectors.fillna(method='ffill')

In [37]:
X = docs_vectors
y = reviews.sentiment.values

In [101]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = \
train_test_split(X,y,test_size=0.25,random_state=20)

In [102]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [103]:
logreg = LogisticRegression()

In [104]:
logreg.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [105]:
y_test_pred = logreg.predict(X_test)

In [106]:
accuracy_score(y_test,y_test_pred)

0.839572192513369